# HeatMap for WoW Token Prices

Map all of the available data in the JSON available from https://wowtoken.info/ for each region to view the data by timeslots for each day of the week

**All times are in UTC**

In [165]:
#!/usr/bin/env python


Populating the interactive namespace from numpy and matplotlib
